In [39]:
from schemathesis import runner, models, loaders
import requests,json
from datetime import timedelta
from hypothesis import Phase
from schemathesis.runner.events import BeforeExecution,AfterExecution

In [40]:
class Environments:
    Prod = {'apikey': 'YJgvtmHEWY39vxITA2fDW8cdrH6s4Epz9bYvlACE' ,'base_url': 'https://api.sentieo.com/v1'}
    Dev =  {'apikey':'N4K02o4YFh1VDbsvxlnAa75V4UO0JLJk2Lwm4f1u' ,'base_url': 'https://pzeiwi7e75.execute-api.us-east-1.amazonaws.com/devv1'}


In [41]:
#checks
def check_for_performance(response, case: models.Case):
    assert response.elapsed < timedelta(milliseconds=10000)
    
def check_for_response_status(response,case: models.Case):
    assert response.status_code == 200 or response.status_code == 204

# AfterExecution event of runner     
def after_run(event,reports):
    if event.result.path not in reports:
        reports[event.result.path] = {}
    if event.result.method not in  reports[event.result.path]:
        reports[event.result.path][event.result.method] = {}   
    reports[event.result.path][event.result.method].update({'status':event.status.name })
    
def get_schema(schema_uri,auth):
    schema = json.loads(requests.get(schema_uri,auth=auth).text)
    schema['servers'] = [{'url': 'https://api.sentieo.com/'}]
    return json.dumps(schema)
    
additional_checks = [check_for_performance, check_for_response_status]
check_for_apis =  { '/documents/get':   [ check_for_performance ] }

In [42]:
#github repo or https://developer.sentieo.com
github_repo = 'https://raw.githubusercontent.com/usernames/sentieodocs/'
# use branch or commit_id(example: e7e9fe5279aedaa3cf68406c308f89b7310c93d4 )
github_branch_or_commit = 'monitorBranch'
#github path or /playground/sentieo.json
github_path = '/play-dev/sentieo.json'

# (username,access_token) for github repo/ (username,pwd) for developer.sentieo.com
auth = ('arvindjangra1996','5435bdc17f8c7b54998b8eba33031fbde8a5dd91')

endpoint = '/documents' # to run documents apis. None for running all.

schema_uri = github_repo+github_branch_or_commit+github_path

reports = {}
runner_config = 'threadpool' or 'inprocess'
env = Environments.Dev
headers = {'x-api-key':env['apikey'] }

schema = get_schema(schema_uri,auth)           
           
events = runner.prepare(
    schema_uri= schema,
    headers=headers,
    loader=loaders.from_file, 
    base_url=env['base_url'],
    hypothesis_deadline=10000,
    checks=additional_checks,
    validate_schema=False,
    endpoint=endpoint,
    hypothesis_phases=[Phase.explicit]
    )

for event in events:
    if isinstance(event,AfterExecution):
        after_run(event,reports)
        
print(reports)

{'/documents/get': {'GET': {'status': 'success'}}, '/documents/search': {'POST': {'status': 'success'}}, '/documents/get-pdf': {'GET': {'status': 'success'}}, '/documents/get-meta': {'GET': {'status': 'success'}}, '/documents/poll': {'GET': {'status': 'success'}}, '/documents/hits': {'GET': {'status': 'success'}}}
